In [240]:
from functools import partial
import numpy as np
import numpy.linalg as la

from scipy.optimize import minimize

from algorithms.algorithm import Algorithm
from distributions.sequence import Sequence
from misc.matrix_geometric_resampling import matrix_geometric_resampling

In [241]:
def mvee(points, tol=0.0001):
    """
    Finds the ellipse equation in "center form"
    (x-c).T * A * (x-c) = 1
    """
    N, d = points.shape
    Q = np.column_stack((points, np.ones(N))).T
    err = tol+1.0
    u = np.ones(N)/N
    while err > tol:
        # assert u.sum() == 1 # invariant
        X = np.dot(np.dot(Q, np.diag(u)), Q.T)
        M = np.diag(np.dot(np.dot(Q.T, la.inv(X)), Q))
        jdx = np.argmax(M)
        step_size = (M[jdx]-d-1.0)/((d+1)*(M[jdx]-1.0))
        new_u = (1-step_size)*u
        new_u[jdx] += step_size
        err = la.norm(new_u-u)
        u = new_u
    c = np.dot(u, points)
    A = la.inv(np.dot(np.dot(points.T, np.diag(u)), points)
               - np.multiply.outer(c, c))/d
    return A, c

def distance_to_ellipsoid_boundary(ellipsoid_H, ellipsoid_center, point: np.ndarray) -> float:
    return (ellipsoid_center - point) @ ellipsoid_H @ (ellipsoid_center - point) - 1

def find_johns(actionset, tolerance=1e-5):
    ellipsoid_H, ellipsoid_center = mvee(actionset, tolerance)

    boundary_points = []
    for index, point in enumerate(actionset):
        if distance_to_ellipsoid_boundary(ellipsoid_H, ellipsoid_center, point) <= tolerance:
            boundary_points.append(index)
    boundary_points = np.array(boundary_points)
    return boundary_points, ellipsoid_center


In [242]:
K = 3
actionset = []
for i in range(K):
    buffer = np.zeros(K, dtype=bool)
    buffer[i] = 1
    buffer[(i+1)%K] = 1
    actionset.append(buffer)
actionset.append(np.ones(K, dtype=bool))
actionset = np.array(actionset)

print(actionset)
print(find_johns(actionset))

[[ True  True False]
 [False  True  True]
 [ True False  True]
 [ True  True  True]]
(array([0, 1, 2, 3]), array([0.75, 0.75, 0.75]))


In [243]:
boundary_points, center = find_johns(actionset)
#boundary_points = np.arange(len(actionset))

boundary_point_matrices = np.zeros((len(boundary_points), K, K))
for i in range(len(boundary_points)):
    boundary_point_matrices[i] = actionset[i].reshape(-1, 1) @ actionset[i].reshape(1, -1)
    print(boundary_point_matrices[i], actionset[i])

def fun(mu):
    new_id = np.zeros((K, K))
    for j in range(len(mu)):
        new_id += K * mu[j] * boundary_point_matrices[j]

    return np.linalg.norm(new_id - np.identity(K))

[[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 0.]] [ True  True False]
[[0. 0. 0.]
 [0. 1. 1.]
 [0. 1. 1.]] [False  True  True]
[[1. 0. 1.]
 [0. 0. 0.]
 [1. 0. 1.]] [ True False  True]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] [ True  True  True]


In [244]:
bnds = []
for i in range(K):
    bnds.append((0, 1))

cons = {'type':'eq', 'fun': lambda x: np.sum(x) - 1}

sol = minimize(fun, x0=np.random.random(K), bounds=bnds, constraints=cons)
print(sol)

     fun: 3.000000000042085
     jac: array([3.99997902, 4.00000364, 4.00001746])
 message: 'Optimization terminated successfully.'
    nfev: 37
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.333331  , 0.33333373, 0.33333527])


In [245]:
solution = sol.x

new_id = np.zeros((K, K))
for j in range(len(solution)):
    print(solution[j], boundary_point_matrices[j])
    new_id += K * solution[j] * boundary_point_matrices[j]

print(new_id)

check_num = 10
x = (np.random.random((K, check_num)) * 2 - 1) * 1e+4

x_new = np.zeros((K, check_num))
for j in range(len(solution)):
    x_new += K * solution[j] * boundary_point_matrices[j] @ x
total = np.linalg.norm(x_new - x, axis=0)
print(total.shape)

print(np.sum(total), np.sum(total) / check_num, np.max(total))

0.33333099895849055 [[1. 1. 0.]
 [1. 1. 0.]
 [0. 0. 0.]]
0.3333337329978814 [[0. 0. 0.]
 [0. 1. 1.]
 [0. 1. 1.]]
0.3333352680436281 [[1. 0. 1.]
 [0. 0. 0.]
 [1. 0. 1.]]
[[1.9999988 0.999993  1.0000058]
 [0.999993  1.9999942 1.0000012]
 [1.0000058 1.0000012 2.000007 ]]
(10,)
120022.02765887424 12002.202765887425 43379.24006579197
